### SF crime data analysis and modeling  
The SFPD provides public access to its well-maintained crime incidents reports to empower use of data, as an important step towards the goal of improving the quality of life and work for SF residents and visitors. In this project I will use historical crime data collected from 2003 to 2018.05 (~200 mill rows of records).

#### The main goal of this project is to:
  - use spark RDD/DataFrame API to perform exploratory data analysis (EDA);
  - develop informative and interactive visualizations to better communicate EDA conclusions;
  - use stats related libraries to model and predict the number of crimes in SF downtown. (requires additional reasonable assumptions)
  
#### The project consists of three parts:
  - PART 0, data preparation, including data download, data import/ load, data preprocessing;
  - PART I, the EDA, the analysis is mainly driven by the 7 questions, focusing on differenct aspects of crime incidents;
  - PART II, time series analysis on number of crime incidents in SF downtown.

##### PS: all codes tested on runtime version: 5.5 LTS (includes Apache Spark 2.4.3, Scala 2.11) and 6.4ML (Apache Spark 2.4.3, Scala 2.11)

### PART 0 - Data Preparation

In [3]:
# Details see https://docs.databricks.com/user-guide/dev-tools/dbutils.html#dbutils-library
# dbutils.library.installPyPI('matplotlib', version='3.1.0') # 
# dbutils.library.restartPython()
'''Library utilities are not available on Databricks Runtime for Machine Learning.'''
# workaround:  in workspace -> create library under Shared -> install on cluster

Out[2]: 'Library utilities are not available on Databricks Runtime for Machine Learning.'

In [4]:
from csv import reader
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
from pyspark.sql.window import Window 
import pyspark.sql.functions as F
import pandas as pd # 0.19.2
import numpy as np # 1.11.1
import seaborn as sb # 0.7.1
import matplotlib.pyplot as plt
import matplotlib as mpl # 1.5.3
import folium # 0.9.1 ==> 0.10.1 as of Mar.22 2020
import warnings
warnings.filterwarnings("ignore",category=UserWarning) # to suppress warnings when importing some libraries
import urllib.request

import os
os.environ["PYSPARK_PYTHON"] = "python3" 
# can be ignored, cluster setting has already specify Environment Variables: PYSPARK_PYTHON=/databricks/python3/bin/python3

In [5]:
# ------------------------------------------------------------------------------------------------------------------------------------------------
# DATA IMPORT
# ------------------------------------------------------------------------------------------------------------------------------------------------

# Download data from SF gov website www.sfgov.org
# urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
# dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
# display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

crime_data_lines = sc.textFile('dbfs:/laioffer/spark_hw1/data/sf_03_18.csv') 
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

# get header --> List<str>
header = df_crimes.first()
print(f'This is what header looks like: \nheader: \n {header}') 

# data without header
crimes = df_crimes.filter(lambda x: x != header) 
crimes.cache()
print('-'*120)

# get the total number of data 
print(f'Now the total crime count is: {crimes.count()}')

# only read first 13 columns for every row of RDD, the rest are redundant or safe to drop due to imput error
createRow = Row(*header[0:13]) 
rdd_rows = crimes.map(lambda x : createRow(*x[0:13])) 

df_opt2 = spark.createDataFrame(rdd_rows)
df_opt2.cache()
# display(df_opt2.limit(10))

# ------------------------------------------------------------------------------------------------------------------------------------------------
# DATA PREPROCESSING（duplication, type conversion, missing values)
# ------------------------------------------------------------------------------------------------------------------------------------------------

# 1. check duplicates
print(f'Check to make sure there are no duplicates: {df_opt2.distinct().count() == df_opt2.count()}.') # both = 2215024 ==> no duplicate rows
# 如果有，df.drop_duplicates(subset=['colName'])

# 2. check schema，identify columns that need type conversion
print(f'The schema of the dataframe is: ')
df_opt2.printSchema()
# Date --> date --> int (year, month, day)
# Time --> timestamp --> int (hour)
# X, Y --> float
# Resolution --> int ( 0 or 1)

# 3. check missing values for every column: 
# 1) Null, 
# 2) empty string or ' ' 
# 3) NaN
missing = df_opt2.select([F.count(F.when((df_opt2[col].isNull())
                                         |(df_opt2[col] == '')
                                         |(df_opt2[col] == ' ')
                                         | F.isnan(df_opt2[col]), col))
                          .alias(col)
                          for col in df_opt2.columns]
                        )
with pd.option_context('display.max_rows', 100, 'display.max_columns', 15):
  print(missing.toPandas())
# display(df_opt2.filter(df_opt2['PdDistrict'] == ''))
# if there are many missing values, we can perform imputation. e.g. fill in missing values in column, say, 'PdDistrict' based on column 'Location'
# yet in this dataset ( ~200 mil rows) , only one observation/ row is missing, we can simply drop this one row:
temp1 = df_opt2.filter(df_opt2['PdDistrict'] != '')

# 4. type conversion based on results from part 2
temp2 = temp1.withColumn('X', F.col('X').cast(DoubleType())) \
        .withColumn('Y', F.col('Y').cast(DoubleType())) \
        .withColumn('Date_', F.to_date(F.col('Date'), 'MM/dd/yyyy')) \
        .withColumn('Hour', F.hour(F.to_timestamp(F.col('Time'), 'HH:mm'))) \
        .withColumn('Resolution', F.when(F.col('Resolution')!='NONE', 1).otherwise(0)) \
        .drop('IncidntNum','Location')
# column 'Date_' is of type Date, to make databricks' buit-in visualization easier (e.g. Line chart requires data in Date type)
df = temp2.withColumn('Year', F.year(F.col('Date_'))) \
        .withColumn('Month', F.month(F.col('Date_'))) \
        .withColumn('Day', F.dayofmonth(F.col('Date_'))) 
print(f'There are in total {df.cache().count()} rows of records in the dataset.')

This is what header looks like: 
header: 
 ['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
------------------------------------------------------------------------------------------------------------------------
Now the total crime count is: 2215024
Check to make sure there are no duplicates: True.
The schema of the dataframe is: 
root
-- IncidntNum: string (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- Date: string (nullable = true)
-- Time: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: string (nullable = true)
-- Y: string (nullable = true)
-- Location: string (nullable = true)
-- PdId: string (nullable = true)

 IncidntNum Category Descript DayOfWeek Date Time PdDistrict \
0 0 0 0 0 0 0 1 

 Resolution Address X Y Location PdId 
0 0 0 0 0 0 0 
There are in total 2215023 rows of records in the dataset.

### PART I - Q1 ~ Q7

#### Q1 question (OLAP): 
##### Counts the number of crimes for different categories

In [8]:
# pyspark.sql.functions as F
q1 = df.groupBy('Category').count().sort(F.desc('count')) 
crimes_pd_df = q1.toPandas()

plt.close('all')
fig, ax = plt.subplots(figsize=(14,5)) # adjust figure size

def format_bar(ax, df, key, value, title_txt, x_txt, y_txt
               , stretch=2.5, width=1.2, align='center', color='SkyBlue', label_rotat=0, label_va='top', label_ha='center', bottom=0):
  labels = df[key]
  data = df[value]
  loc = np.arange(len(labels)) #use loc to mark bars location(in case matplotlib.__version__ < 2.0.0, pass Pandas.Series in ax.bar()will raise error)
  loc_stretch = [stretch*i for i in loc] # space out bars
  bars = ax.bar(loc_stretch, data, width=width, align=align, color=color, bottom=bottom) #
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.set_xticks(loc_stretch)
  ax.set_xticklabels(labels=labels, rotation=label_rotat, fontsize='8', va=label_va, ha=label_ha)
  # horizontalalignment {'center', 'right', 'left'}
  # verticalalignment {'center', 'top', 'bottom', 'baseline', 'center_baseline'}
  ax.get_yaxis().set_major_formatter(
      mpl.ticker.FuncFormatter(lambda x, p : str(x)[:-5]+' K' if x!=0 else '0')) # or format(int(x), ',')
  ax.title.set_text(title_txt)
  ax.set_xlabel(x_txt)
  ax.set_ylabel(y_txt)
  return bars

bars = format_bar(ax, crimes_pd_df, 'Category', 'count', 'Number of Crimes in SF by Category (2003 - 2018.05)', 'Categories of crimes', '# of crimes'
                 , label_rotat=-30, label_ha='left')

# Attach a text label above each bar displaying its height
def bar_autolabel(ax, bars, ha='center', va='bottom', rotation=0):
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., 1.05*height, f'{format(int(height), ",")}', ha=ha, va=va, rotation=rotation)
        
bar_autolabel(ax, bars, ha='left', rotation=30)
        
plt.tight_layout() # stretch out plot to fit the figure
display(fig)

#### Q2 question (OLAP)
Counts the number of crimes for different district, and visualize your results

In [10]:
# pyspark.sql.functions as F
q2_df_1 = df.groupBy('PdDistrict').count().sort(F.desc('count')).withColumnRenamed('count','# of incidents').toPandas()
q2_df_2 = df.groupBy('Category','PdDistrict').count().sort(F.desc('count')).withColumnRenamed('count','# of incidents')

In [11]:
plt.close('all')
fig, ax = plt.subplots(figsize=(14,3))

bars1 = format_bar(ax, q2_df_1, 'PdDistrict', '# of incidents', '# of incidents by districts', 'PdDistrict', '# of incidents')
bar_autolabel(ax, bars1, ha='center')

plt.tight_layout()
display(fig)

In [12]:
display(q2_df_2) 
# use '100% stacked' option in 'Bar Chart', with:
# Keys: 'PdDistrict'
# Series groupings: 'Category'
# Values: SUM('# of incidents')

Category,PdDistrict,# of incidents
LARCENY/THEFT,SOUTHERN,113342
LARCENY/THEFT,NORTHERN,80988
LARCENY/THEFT,CENTRAL,71948
OTHER OFFENSES,SOUTHERN,51403
NON-CRIMINAL,SOUTHERN,49821
LARCENY/THEFT,MISSION,48960
OTHER OFFENSES,MISSION,47363
OTHER OFFENSES,BAYVIEW,40805
DRUG/NARCOTIC,TENDERLOIN,38211
OTHER OFFENSES,INGLESIDE,32668


##### EN
- In terms of total # of crime incidents:
  - Southern, Mission and Northern have the most number of crime incidents.
- In terms of category of crimes in different districts:
  - In almost all districts, majority of the crimes fall into 'Larceny' category (if not consider 'other offsenses') 
  - The district Tenderloin is in sharp contrast with other areas, drug related crimes is the dominant crime type, therefore the anti-drug effort might be further strengthened.

##### CN
- 通过分析不同区域案件整体数量:
  - Southern, Mission 和 Northern 为案件数量最多的三个区域
- 通过分析不同区域各类案件占比:
  - 如果忽略“其他类型案件”(other offenses)的占比, 几乎在所有区域, Larceny 都是主要的案件类型, 警力的主要关注对象应该focus Larceny;
  - Tenderloin 区域的毒品相关案件在整体案件的占比明显大于其他区域, 甚至超过Larceny成为该区域主要案件类型, 该区域缉毒力度应该进一步加强;

#### Q3 question (OLAP)
Count the number of crimes each "Sunday" at "SF downtown".   

SF downtown is defiend  via the range of spatial location. For example, a rectangle to define the SF downtown, or a cicle with center as well. Write UDF function to filter data which are located inside certain spatial range. For UDF, see: https://changhsinlee.com/pyspark-udf/

***Main Takeaways***:
- return type has to be specified in udf; it's always safer to force type conversion INSIDE python function
- when output is a composite, use ArrayType(otherType()); 
  - for mixed type, use StructType([StructField('name', someType(), nullable = True/False), StructField(similar stuff)])
- Common error:
  - Py4JJavaError: where spark does not recognized the return type, say, np.ndarray() --> convert type in python function( i.e. np.ndarray().tolist() )
  - Slowness: the udf may not be excuted in a parallelized way due to smaller size of dataframe --> partition the df( i.e. df_repartitioned = df.repartition(100))

In [15]:
# I'll simply define financial district as the area covered by an ellipse, with its center at 37.7946° N, 122.3999° W, major axis extending 0.005 degree along longitude(~ 426 m), minor axis extending 0.005 degree along latitude (~ 555m), this configuration could easily be adjusted.

@udf('Boolean')
def within_circle(x, y):
  return (x + 122.3999)**2 + (y - 37.7946)**2 < 0.000025
# using the decorator ('@udf') is just a syntactic sugar for: within_circle = udf(within_circle, BooleanType())

q3 = (df.select('Category', 'DayOfWeek', 'X', 'Y', 'PdId', 'PdDistrict', 'Year', 'Month', 'Day', 'Date_')
      .where((df.DayOfWeek == "Sunday") & within_circle(df.X, df.Y)) # see preprocessing in PART 0 for the rationale behind generating column 'Date_'
     )
display(q3.groupby('Year', 'Month', 'Day', 'Date_').count().sort(['Year', 'Month', 'Day'], ascending=[1, 1, 1]).withColumnRenamed('count', 'Total # of crimes'))
# use 'Line Chart', with:
# Keys: 'Date_'
# Values: 'Total # of crimes'

Year,Month,Day,Date_,Total # of crimes
2003,1,5,2003-01-05,2
2003,1,12,2003-01-12,4
2003,1,19,2003-01-19,1
2003,1,26,2003-01-26,4
2003,2,2,2003-02-02,4
2003,2,9,2003-02-09,1
2003,2,16,2003-02-16,3
2003,2,23,2003-02-23,8
2003,3,2,2003-03-02,2
2003,3,9,2003-03-09,5


##### EN

Over the years, the total number of crime incidents on 'Sundays' in 'SF downtown' seems to have an upward trend, and it is more pronounced during 2010 - 2016;  
In PART II, time series analysis will follow up on and try to quantify this idea through trend fitting.

##### CN
虽然没有进一步做趋势拟合, SF downtown的犯罪数量似乎呈现出一些上升的趋势, 2010年到2016年这一段时间尤为明显
Q8 会对SF downtown的数据做时间序列的follow-up分析

#### Q4 question (OLAP)
Analyse the number of crime in each month of 2015, 2016, 2017, 2018. Then, give your insights for the output results.

In [18]:
# pyspark.sql.functions as F
# Either register udf by: is_year = udf(lambda x : x == 2015 or x == 2016 or x == 2017 or x == 2018, BooleanType()), then apply is_year(df.Year) 
# OR use column.isin() function, note the * syntax 

q4 = (df.select(df.Year, df.Month, df.PdDistrict, df.Category, df.PdId, df.Resolution)
      .where(df.Year.isin(*[2015,2016,2017,2018]))
      .groupby(df.Year, df.Month, df.Category, df.PdDistrict, df.Resolution).count()
      .sort(df.Year, df.Month, df.Category, df.PdDistrict, df.Resolution)
     )
q4.cache()

Out[21]: DataFrame[Year: int, Month: int, Category: string, PdDistrict: string, Resolution: int, count: bigint]

In [19]:
# from pyspark.sql.window import Window

window_q4 = Window.partitionBy('Year', 'Month').orderBy(F.desc('NumOfIndicidnt'))

ans = (q4.filter('Year != 2018 OR Month != 5') # see note2 bellow for reasons to exclude data from 2018.05
       .groupby(q4.Year, q4.Month, q4.Category).sum('count').withColumnRenamed('sum(count)', 'NumOfIndicidnt')
       .withColumn('rnk', F.row_number().over(window_q4))
       .filter('rnk <= 5')
       .sort('Year', 'Month', 'rnk')
      )
display(ans)
# use 'Line(deprecated)', with:
# Keys: 'Year', 'Month'
# Series groupings: 'Category'
# Values: 'NumOfIncident'

Year,Month,Category,NumOfIndicidnt,rnk
2015,1,LARCENY/THEFT,3715,1
2015,1,OTHER OFFENSES,1809,2
2015,1,NON-CRIMINAL,1676,3
2015,1,ASSAULT,1113,4
2015,1,VANDALISM,616,5
2015,2,LARCENY/THEFT,3284,1
2015,2,OTHER OFFENSES,1667,2
2015,2,NON-CRIMINAL,1554,3
2015,2,ASSAULT,1011,4
2015,2,WARRANTS,604,5


##### EN
1. Category 'Larceny' is the major type of crime in almost all districts in SF, hence naturally, the fluctuation of the total number of crime incidents are dominated by that of the number of crimes identified as 'Larceny'.
2. All major categories of crime have been taking on a downward trend since 2015, except for Larceny, which 


##### CN
1. 因为larceny发生的数量远高于其他类型的crime，因此SF整体的crime发生次数的波动趋势由larceny主导。
2. 15年各类crime基本都呈现出稳中有降的趋势，except for larceny在年初出现了4个月左右的波动；
3. 从16年开始，除了Larceny以外的各类crime 都在逐步缓慢减少，并在18年初达到近3年来最低水平；
4. larceny从16年初开始逐渐成上升趋势一直到17年10月达到顶峰，才开始进入较快下降趋势，同样在18年初达到了3年以来最低水平；

注1: 因为2018年5月的数据只有15天,不能代表整月的crime情况,因此剔除出展示结果,防止不必要的误导.  
注2: 使用window function之后, 因为并非每个partition top5 类型都是稳定的, 所以有的月份, 末位的类型显示为零, 但并不代表真实情况, 只是filter的结果而已, interpret时需要注意一下

#### Q5 question (OLAP)
Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15. Then, give your travel suggestion to visit SF.

In [22]:
# pyspark.sql.functions as F
# filter out the day we are interested in
udf_q5 = udf(lambda x, y, z : x == 12 and y == 15 and (z == 2015 or z == 2016 or z == 2017), BooleanType())

# preprocess
q5 = df.select('Year', 'Month', 'Day', 'Hour', 'Category', 'Pdid') \
        .where(udf_q5(df.Month, df.Day, df.Year)) \
        .groupBy('Year', 'Month', 'Day', 'Hour', 'Category').count()

ans = q5.select('Year', 'Month', 'Day', 'Hour', 'Category', F.col('count').alias('NumOfIncidnt')) \
        .sort('Year', 'Month', 'Day', 'Hour', 'Category')
display(ans)
# use 'Line Chart', with:
# Keys: 'Hour'
# Series groupings: 'Year', 'Month', 'Day'
# Values: 'NumOfIncident'

Year,Month,Day,Hour,Category,NumOfIncidnt
2015,12,15,0,BURGLARY,1
2015,12,15,0,FRAUD,1
2015,12,15,0,LARCENY/THEFT,3
2015,12,15,0,NON-CRIMINAL,1
2015,12,15,0,OTHER OFFENSES,7
2015,12,15,0,ROBBERY,1
2015,12,15,0,SUSPICIOUS OCC,1
2015,12,15,1,ASSAULT,1
2015,12,15,1,DRUNKENNESS,1
2015,12,15,1,OTHER OFFENSES,1


- 2015, 2016以及2017年三年间12月15日这一天每一个小时的案件数量呈现出几乎一致的趋势, 整体上, 中午12点以及晚间7点, 案件数量呈现出一个高峰, 而在上午, 下午1点到5点, 晚间8点到9点案件发生数量较少;
- 从三年的变化上看, 也许是由于中午12点案件的高发导致警力分配以及重点关注时刻的改变(仅仅是speculation, 尚未有数据支撑), 中午12点的案件发生数量逐年减少, 到2017年12月15日, 中午12点到下午3点事实上已经成为案件数量较少的时刻, 反而是上午10点, 晚间7点及晚间10点成为高峰;
- 作为出行建议, 白天游览SF, 建议中午12点到下午3点, 避开整段上午时间以及午后3点以后; 晚间则可以在8点到9点, 避开其他时段

#### Q6 question (OLAP)
(1) Step1: Find out the top-3 danger disrict  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) give your advice to distribute the police based on your analysis results.

In [25]:
# Assumption: 按照案件数量(不区分类型)来定义“dangerous”
# Assumption: 主要类型的案件dominant了案件数量, 所以每个hour只取类型最多的5类案件进行分析和visualize

# 1) 由Q2得到: 最危险的三个区域分别是 Southern, Mission 和 Northern

# 2) filter out the data we are interested in

# define a window and use the window function to tease out the top 5 frequent category of crimes
window_q6 = Window.partitionBy('Hour').orderBy(F.desc('NumOfIncidnt'))

q6_1 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'SOUTHERN') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

q6_2 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'MISSION') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

q6_3 = df.select(df.PdId, df.Hour, df.Category, df.PdDistrict) \
        .filter(df.PdDistrict == 'NORTHERN') \
        .groupBy('Category', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('rnk', F.row_number().over(window_q6)) \
        .filter('rnk <= 5') \
        .sort('Hour') 

In [26]:
display(q6_1) # Southern
# use 'Line Chart', with:
# Keys: 'Hour'
# Series groupings: 'Category'
# Values: 'NumOfIncident'

Category,Hour,NumOfIncidnt,rnk
LARCENY/THEFT,0,4163,1
OTHER OFFENSES,0,2947,2
NON-CRIMINAL,0,2414,3
ASSAULT,0,1714,4
FRAUD,0,1174,5
NON-CRIMINAL,1,1243,4
LARCENY/THEFT,1,2577,1
ASSAULT,1,1506,2
OTHER OFFENSES,1,1415,3
VANDALISM,1,705,5


In [27]:
display(q6_2) # Mission
# use 'Line Chart', with:
# Keys: 'Hour'
# Series groupings: 'Category'
# Values: 'NumOfIncident'

Category,Hour,NumOfIncidnt,rnk
OTHER OFFENSES,0,2754,1
LARCENY/THEFT,0,2554,2
ASSAULT,0,1801,3
NON-CRIMINAL,0,1538,4
VANDALISM,0,868,5
ROBBERY,1,650,5
LARCENY/THEFT,1,1822,1
ASSAULT,1,1645,2
OTHER OFFENSES,1,1521,3
NON-CRIMINAL,1,964,4


In [28]:
display(q6_3) # Northern
# use 'Line Chart', with:
# Keys: 'Hour'
# Series groupings: 'Category'
# Values: 'NumOfIncident'

Category,Hour,NumOfIncidnt,rnk
LARCENY/THEFT,0,3376,1
OTHER OFFENSES,0,2048,2
NON-CRIMINAL,0,1271,3
ASSAULT,0,1209,4
VANDALISM,0,761,5
NON-CRIMINAL,1,751,4
LARCENY/THEFT,1,2247,1
ASSAULT,1,1188,2
OTHER OFFENSES,1,1024,3
VANDALISM,1,602,5


In [29]:
# define another window to tease out sub-categories
window_q6_2 = Window.partitionBy('Hour', 'Category').orderBy(F.desc('NumOfIncidnt'))

q6_2_2 = df.select(df.PdId, df.Hour, df.Category, df.Descript, df.PdDistrict) \
        .filter(df.PdDistrict == 'MISSION') \
        .groupBy('Category', 'Descript', 'Hour').count().withColumnRenamed('count','NumOfIncidnt') \
        .withColumn('sub_rnk', F.row_number().over(window_q6_2)) \
        .sort('Hour','sub_rnk') 

#用category粒度的数据 left join descript粒度的数据,得到top3大类案件下的top3细分类别案件

ans = q6_2.drop('NumOfIncidnt').join(q6_2_2, ['Category', 'Hour'], how = 'left') \
            .filter('rnk <=3 AND sub_rnk <= 3') \
            .select('Category','Descript','Hour','NumOfIncidnt','rnk','sub_rnk') \
            .sort('Hour','rnk','sub_rnk') 
display(ans)
# use 'Line Chart', with:
# Keys: 'Hour'
# Series groupings: 'Category','Descript'
# Values: 'NumOfIncident'

# 注: Join 条件使用 (q6_2.Category == q6_2_2.Category) & (q6_2.Hour == q6_2_2.Hour) 会导致duplicate columns
# https://kb.databricks.com/data/join-two-dataframes-duplicated-columns.html

Category,Descript,Hour,NumOfIncidnt,rnk,sub_rnk
OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",0,562,1,1
OTHER OFFENSES,TRAFFIC VIOLATION,0,355,1,2
OTHER OFFENSES,TRAFFIC VIOLATION ARREST,0,249,1,3
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,0,732,2,1
LARCENY/THEFT,PETTY THEFT OF PROPERTY,0,387,2,2
LARCENY/THEFT,GRAND THEFT OF PROPERTY,0,241,2,3
ASSAULT,BATTERY,0,521,3,1
ASSAULT,THREATS AGAINST LIFE,0,216,3,2
ASSAULT,AGGRAVATED ASSAULT WITH A DEADLY WEAPON,0,203,3,3
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,1,478,1,1


In [30]:
# to better understanding the result, we need detailed decriptions of category: 'NON-CRIMINAL','OTHER-OFFENSES' and 'LARCENY/THEFT'
dim_category = ans.select('Category', 'Descript') \
                  .filter(F.col('Category').isin(['NON-CRIMINAL', 'OTHER OFFENSES', 'LARCENY/THEFT'])) \
                  .drop_duplicates(['Category', 'Descript']) \
                  .sort('Category', 'Descript')
display(dim_category)

Category,Descript
LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO
LARCENY/THEFT,GRAND THEFT OF PROPERTY
LARCENY/THEFT,PETTY THEFT FROM A BUILDING
LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO
LARCENY/THEFT,PETTY THEFT OF PROPERTY
LARCENY/THEFT,PETTY THEFT SHOPLIFTING
NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED"
NON-CRIMINAL,FOUND PROPERTY
NON-CRIMINAL,LOST PROPERTY
OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED"


Assumption1: 按照案件数量(不区分类型)来定义“dangerous”  
Assumption2: 主要类型的案件dominant了案件数量, 所以每个hour只取类型最多的5类案件进行分析和visualize
Assumption3: Mission区不同于其他区域,对案件类别细分时,针对top3 大类分别提取出top3 细分类别
- 由Q2得到: 最危险的三个区域分别是 Southern, Mission 和 Northern;
- 可以看到三个区域案件集中在中午12点到晚间8点, 以晚间7点左右为最高点; 
- Southern和Northern区的主导案件类型都是Larceny, 细分类别为Grand theft from locked auto,即砸车盗窃;
- Mission区略有不同, 砸车盗窃发生数量明显低于其他两区, 以高点为例, Southern 约 9k, Northern 约 6k, 而Mission 仅约3k (累计发生量);
- Mission区‘Other offenses’和‘non-criminal’两类案件均多余其他两区, 查看细分类别, other offenses中以 **traffic violation**为主导, non-crinimal 以**lost property** 为主导;
- 建议:
  - Southern区和Northern区应该重点关注砸车盗窃案件, 增强此类案件多发区域(需要进一步分析Address得到,可能是residential,也可能是commercial)的patrol;
  - Mission除了关注砸车盗窃案件外,对于交通违规案件,针对事故多发地段增强警力部署,对于lost property 也进行对应位置的警力部署,可以结合加强safe neighborhood的宣传等

#### Q7 question (OLAP)
For different category of crime, find the percentage of resolution. Based on the output, give your hints to adjust the policy.

In [33]:
# 政策建议应该结合时间这个因素, 分析各类案件解决与否随着时间的变化趋势
# 考虑实效性, 仅取最近的3年(2015-2018)数据
# 另: 因为2018年5月的数据只有15天,不能代表整月的crime情况,因此剔除出展示结果,防止不必要的误导

q7_1 = df.select(df.Category, df.Resolution, df.Year, df.Month, df.PdId) \
        .filter(df.Year.isin(*[2015, 2016, 2017, 2018])) \
        .filter('Year != 2018 OR Month != 5') \
        .groupBy('Category', 'Resolution').count().withColumnRenamed('count', 'NumOfIndicidnt') \
        .withColumn('% resolved', F.col('NumOfIndicidnt')/F.sum('NumOfIndicidnt').over(Window.partitionBy('Category'))) \
        .filter('Resolution = 0') \
        .drop('Resolution', 'NumOfIndicidnt') \
        .sort(F.col('% resolved').desc()) 
display(q7_1)

Category,% resolved
SUICIDE,0.9561752988047809
LARCENY/THEFT,0.9556050928708194
RUNAWAY,0.9369951534733441
BAD CHECKS,0.9333333333333333
RECOVERED VEHICLE,0.9215189873417722
VEHICLE THEFT,0.9135733556609917
SUSPICIOUS OCC,0.8993207308725185
EXTORTION,0.8959537572254336
MISSING PERSON,0.8878198567041965
VANDALISM,0.8820466221647503


In [34]:
window_q7 = Window.partitionBy('Year', 'Month', 'Category') # 方便计算 resolution percentage

q7_2 = df.select(df.Category, df.Resolution, df.Year, df.Month, df.PdId) \
        .filter(df.Year.isin(*[2015, 2016, 2017, 2018])) \
        .filter('Year != 2018 OR Month != 5') \
        .groupBy('Category', 'Year', 'Month', 'Resolution').count().withColumnRenamed('count', 'NumOfIndicidnt') \
        .withColumn('% resolved', F.col('NumOfIndicidnt')/F.sum('NumOfIndicidnt').over(window_q7)) \
        .filter('Resolution = 0') \
        .drop('Resolution', 'NumOfIndicidnt') \
        .sort(F.col('% resolved').desc()) \
        .filter(F.col('Category').isin(*['LARCENY/THEFT', 'OTHER OFFENSES', 'NON-CRIMINAL', 'ASSAULT', 'VEHICLE THEFT', 'DRUG/NARCOTIC', 'VANDALISM','WARRANTS'])) \
        .sort('Year', 'Month')
display(q7_2)
# use 'Line(Deprecated)', with:
# Keys: 'Year', 'Month'
# Series groupings: 'Category'
# Values: '% resolved'

Category,Year,Month,% resolved
ASSAULT,2015,1,0.6082659478885895
WARRANTS,2015,1,0.09609120521172639
OTHER OFFENSES,2015,1,0.32780541735765617
DRUG/NARCOTIC,2015,1,0.10268948655256724
LARCENY/THEFT,2015,1,0.9429340511440107
NON-CRIMINAL,2015,1,0.8096658711217184
VEHICLE THEFT,2015,1,0.9092409240924092
VANDALISM,2015,1,0.8652597402597403
LARCENY/THEFT,2015,2,0.9482338611449452
DRUG/NARCOTIC,2015,2,0.1111111111111111


- based on 近三年的数据,SUICIDE 的破案率最高, 其后依次是 LARCENY/THEFT, RUNAWAY, BAD CHECKS, RECOVERED VEHICLE, 排名前五的类型破案率在90%以上;
- DUI案件破案率垫底,仅有不足5%;
- 从近三年破案率呈现出的趋势来看, 主要类型案件的破案率基本维持稳定,没有明显的变化;  
- 对于警方的建议:
  - DUI的危险性较大,应该增强patrol的力度,
  - 可以考虑开通一些市民举报等参与破案的途径,增加DUI类案件的破案率

注: ***主要类型***, 按照Q1所显示的记录在案最多的8类犯罪类型,('LARCENY/THEFT', 'OTHER OFFENSES', 'NON-CRIMINAL', 'ASSAULT', 'VEHICLE THEFT', 'DRUG/NARCOTIC', 'VANDALISM','WARRANTS')

Part 02 see